<a href="https://colab.research.google.com/github/maverick98/Coursera/blob/master/mfml2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from functools import reduce

In [12]:
def create_random_invertible_matrix(n=5):
    A = np.random.randint(low=0,high=10,size=(n, n))
    mx = np.sum(np.abs(A), axis=1)
    np.fill_diagonal(A, mx)
    A=A.astype(float)
    return A
def transpose(A):
    n=A.shape[0]
    A_T=np.zeros((n,n))
    for i in range(n):
        A_T[i]=A[:,i]
    return A_T
def multiply(A,B):
    if A.shape[1] != B.shape[0]:
       print("Illegal multiplication")
       return
    m=A.shape[0]
    n=A.shape[1]
    p=B.shape[1]
    C=np.zeros((m,p))
    for i in range(m):
        for k in range(p):
            for j in range(n):
                C[i][k]+=A[i][j]*B[j][k]

    return C

def create_random_symmetric_positive_definite_matrix(n):
    A=create_random_invertible_matrix(n)
    A_T=transpose(A)
    return multiply(A,A_T)

def create_identity_matrix(n):
     I=np.zeros((n,n))
     for i in range(n):
         I[i][i]=1
     return I



#Elementary row operations
def const_multiple(A,i,c):
    A[i]*=c
def exchange_rows(A,i,j):
    A[[i,j]] = A[[j,i]]
def add_const_row_multiple(A,i,j,c):
    A[i]+=A[j]*c

def find_pivot_row(A, curr_pivot_row, col):
    rows = A.shape[0]
    for row in range(curr_pivot_row, rows):
        if A[row, col] != 0.0:
            return row
    return None




def convert_pivot_row_column_values_to_zero_below(A,pivot_row,col,elementary_matrices):
    n=A.shape[0]
    rows = A.shape[0]

    for row in range(pivot_row+1, rows):
        const_multiplier=A[row][col]/A[pivot_row][col]
        elementary_matrix=create_identity_matrix(n)
        elementary_matrix[row][col]=const_multiplier
        elementary_matrices.append(elementary_matrix)
        #print('subtracting {}*A[{}] from A[{}]'.format(const_multiplier, pivot_row,row))
        A[row] -=const_multiplier*A[pivot_row]
    return elementary_matrices

def calculate_row_echelon_form(A):
    n=A.shape[0]

    pivot_row_cols=[]
    pivot_cols=[]
    non_pivot_cols=[]
    rows=A.shape[0]
    cols=A.shape[1]
    elementary_matrices=[]
    permutation_matrices=[]


    curr_pivot_row=0
    for col in range(cols):
        nonzero_row = find_pivot_row(A, curr_pivot_row, col)
        if nonzero_row is not None:
            if curr_pivot_row != nonzero_row:
                exchange_rows(A,curr_pivot_row,nonzero_row)
                perm_matrix=create_identity_matrix(n)
                exchange_rows(perm_matrix,curr_pivot_row,nonzero_row)
                permutation_matrices.append(perm_matrix)
                perm_matrix_inverse=transpose(perm_matrix)
                elementary_matrices.append(perm_matrix_inverse)
            pivot_row_cols.append((curr_pivot_row,col))
            convert_pivot_row_column_values_to_zero_below(A,curr_pivot_row,col,elementary_matrices)
            curr_pivot_row+=1
        else:
            print('Stopping...')
            if len(elementary_matrices) == 0:
               elementary_matrices.append(create_identity_matrix(n))
            return elementary_matrices,permutation_matrices


    return elementary_matrices,permutation_matrices

def multiply_all_matrices(matrices):
    n=matrices[0].shape[0]
    I=create_identity_matrix(n)
    #product = reduce(lambda x, y: multiply(x,y), matrices,I)
    product=I
    for matrix in matrices:
        product=multiply(product,matrix)
    return product

In [13]:
def calculate_LU_Decomposition(A):
    print('Input matrix is ')
    print(A)
    print('---------')
    elementary_matrices,permutation_matrices=calculate_row_echelon_form(A)
    print('U is')
    print(A)
    print('-----')
    L=multiply_all_matrices(elementary_matrices)
    L_modified=L
    print('------')
    if len(permutation_matrices) >0:

       for permutation_matrix in permutation_matrices:
           L_modified=multiply(permutation_matrix,L_modified)

    L=L_modified
    print("L is")
    print(L)
    print('------')

    if len(permutation_matrices) >0:
        print('perm matrices are {}'.format(len(permutation_matrices)))
        P=multiply_all_matrices(permutation_matrices)
        print(P)
        print('**********')


    #print('Elementary matrices are')
    #for elementary_matrix in elementary_matrices:
    #    print(elementary_matrix)
    #    print('***')

1

In [14]:
A = np.array([[ 1 ,1,1 ], [1,1,3], [2,5,8]])
A=A.astype(float)
calculate_LU_Decomposition(A)


Input matrix is 
[[1. 1. 1.]
 [1. 1. 3.]
 [2. 5. 8.]]
---------
U is
[[1. 1. 1.]
 [0. 3. 6.]
 [0. 0. 2.]]
-----
------
L is
[[1. 0. 0.]
 [2. 1. 0.]
 [1. 0. 1.]]
------
perm matrices are 1
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
**********


In [15]:
A = np.array([[ 10 ,45,0 ], [10,34,0], [10,118,0]])
A=A.astype(float)
calculate_LU_Decomposition(A)


Input matrix is 
[[ 10.  45.   0.]
 [ 10.  34.   0.]
 [ 10. 118.   0.]]
---------
Stopping...
U is
[[ 10.  45.   0.]
 [  0. -11.   0.]
 [  0.   0.   0.]]
-----
------
L is
[[ 1.          0.          0.        ]
 [ 1.          1.          0.        ]
 [ 1.         -6.63636364  1.        ]]
------


In [16]:
A = np.array([[ 10 ,0,45 ], [10,0,34], [10,0,118]])
A=A.astype(float)
calculate_LU_Decomposition(A)


Input matrix is 
[[ 10.   0.  45.]
 [ 10.   0.  34.]
 [ 10.   0. 118.]]
---------
Stopping...
U is
[[ 10.   0.  45.]
 [  0.   0. -11.]
 [  0.   0.  73.]]
-----
------
L is
[[1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
------


In [17]:
A = np.array([[ 0,10,45 ], [0,10,34], [0,10,118]])
A=A.astype(float)
calculate_LU_Decomposition(A)

Input matrix is 
[[  0.  10.  45.]
 [  0.  10.  34.]
 [  0.  10. 118.]]
---------
Stopping...
U is
[[  0.  10.  45.]
 [  0.  10.  34.]
 [  0.  10. 118.]]
-----
------
L is
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
------


In [18]:
A=create_random_symmetric_positive_definite_matrix(n=3)
A

array([[285., 163., 129.],
       [163., 467., 207.],
       [129., 207., 243.]])

In [24]:
calculate_LU_Decomposition(A)

Input matrix is 
[[285.         163.         129.        ]
 [  0.         373.7754386  133.22105263]
 [  0.           0.         137.12787489]]
---------
U is
[[285.         163.         129.        ]
 [  0.         373.7754386  133.22105263]
 [  0.           0.         137.12787489]]
-----
------
L is
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
------


In [84]:
k=1
for i in range(k):
    print(i)

0


In [134]:
def cholesky_decomposition(A):
    n=A.shape[0]
    L=np.zeros((n,n)).astype(float)
    for i in range(n):
        for j in range(i+1):
            if i==j:
               l_sum=0
               for k in range(j):
                    l_sum+=L[j][k]*L[j][k]
               L[i][j]=np.sqrt(A[i][j]-l_sum)
            else:
                l_sum=0
                for k in range(j):
                    l_sum+=L[i][k]*L[j][k]
                L[i][j]= (A[i][j]-l_sum)/L[j][j]
    return L,transpose(L)


In [140]:
A = np.array([[ 4,12,-16 ], [12,37,-43], [-16,-43,98]])
A=A.astype(float)
#calculate_LU_Decomposition(A)
L,L_T=cholesky_decomposition(A)
L,L_T

(array([[ 2.,  0.,  0.],
        [ 6.,  1.,  0.],
        [-8.,  5.,  3.]]),
 array([[ 2.,  6., -8.],
        [ 0.,  1.,  5.],
        [ 0.,  0.,  3.]]))